### Invoke SageMaker Endpoint from outside of AWS environment using SageMaker SDK

Model used: XGBoost Bike Rental Prediction Trained in the XGBoost Lectures

This example uses the IAM user: ml_user_påredict


In [1]:
# Install Sagemker
!pip install --upgrade sagemaker

     |████████████████████████████████| 1.5 MB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 294 kB 51.1 MB/s eta 0:00:01
     |████████████████████████████████| 12.4 MB 74.0 MB/s eta 0:00:01
  Using cached urllib3-2.2.1-py3-none-any.whl (121 kB)
     |████████████████████████████████| 57 kB 10.8 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 12.0 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 72.3 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 485 kB/s  eta 0:00:01
     |████████████████████████████████| 17.3 MB 76.9 MB/s eta 0:00:01
  Using cached PyYAML-6.0.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (736 kB)
     |████████████████████████████████| 345 kB 44.9 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 31.3 MB/s eta 0:00:01
     |████████████████████████████████| 116 kB 32.2 MB/s eta 0:00:01
     |████████████████████████████████| 56 kB 11.2 MB/s eta 0:00:01
ERROR: botocore 1.34.52 ha

In [2]:
! pip list

Package                   Version             
------------------------- --------------------
anyio                     4.3.0               
argon2-cffi               23.1.0              
argon2-cffi-bindings      21.2.0              
asttokens                 2.4.1               
async-lru                 2.0.4               
attrs                     23.2.0              
Automat                   0.8.0               
awscli                    1.32.11             
Babel                     2.14.0              
backcall                  0.1.0               
beautifulsoup4            4.12.3              
bleach                    6.1.0               
blinker                   1.4                 
boto3                     1.34.52             
botocore                  1.34.52             
certifi                   2019.11.28          
cffi                      1.16.0              
chardet                   3.0.4               
charset-normalizer        3.3.2               
Click        

In [3]:
!pip install --upgrade boto3

     |████████████████████████████████| 139 kB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 12.2 MB 35.8 MB/s eta 0:00:01
  Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
ERROR: awscli 1.32.11 has requirement botocore==1.34.11, but you'll have botocore 1.34.98 which is incompatible.
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.1
    Uninstalling urllib3-2.2.1:
      Successfully uninstalled urllib3-2.2.1
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.52
    Uninstalling botocore-1.34.52:
      Successfully uninstalled botocore-1.34.52
  Attempting uninstall: boto3
    Found existing installation: boto3 1.34.52
    Uninstalling boto3-1.34.52:
      Successfully uninstalled boto3-1.34.52


In [4]:
! pip install --upgrade awscli

     |████████████████████████████████| 4.5 MB 3.1 MB/s eta 0:00:01
  Attempting uninstall: awscli
    Found existing installation: awscli 1.32.11
    Uninstalling awscli-1.32.11:
      Successfully uninstalled awscli-1.32.11


In [5]:
import boto3
import sagemaker
import math
import dateutil
import re

# SDK 2 serializers and deserializers
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/saravanan/.config/sagemaker/config.yaml


In [15]:
# Import s3 bucket name from .env file

import os
env_vars = !cat ./.env
for var in env_vars:
    key, value = var.split('=')
    os.environ[key]=value

In [17]:
endpoint_name = os.environ['ENDPOINT_NAME']
profile_name = os.environ['PROFILE_NAME']
#print(endpoint_name)

In [20]:
# Establish a session with AWS 
# specify credentials and region to be used for this session
# We will use a ml_user_predict credentials that has limited preivileges

boto_session = boto3.Session(profile_name=profile_name, region_name='us-east-1')

In [21]:
sess = sagemaker.Session(boto_session=boto_session)

In [ ]:
# Create a predictor and point to an existing endpoint

# Get predictor using SageMaker SDK
# specify your endpoint name

predictor = sagemaker.predictor.Predictor(endpoint_name=endpoint_name, s